In [1]:
import decisiontree as dtree
import numpy as np
import time
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("DTREE-SUSY").getOrCreate()
sc = spark.sparkContext


In [2]:
# 1. Leemos el dataset 
rdd = sc.textFile('../datasets/SUSY.csv')
from pyspark.mllib.linalg import DenseVector
from pyspark.mllib.regression import LabeledPoint
def append_ret(a,b):
    a.append(b)
    return a
# 2. Separamos y ponemos la etiqueta al final en vez de al principio
rdd = rdd.map(lambda line: line.split(','))
rdd = rdd.map(lambda line: append_ret(line[1:], line[0]))
rdd = rdd.map(lambda line: DenseVector(line))
rdd = rdd.repartition(12)

In [3]:
def gpu_work(max_depth=4, min_samples_per_node=1):
    def _gpu_work(data):
        # 1. Procesamos el dataset
        inp = np.asarray(list(data), dtype=np.float32)
        return [dtree.train_tree(inp, max_depth, min_samples_per_node)]    
    return _gpu_work

def cpu_work(max_depth=4, min_samples_per_node=1):
    def _cpu_work(data):
        # 1. Procesamos el dataset
        inp = np.asarray(list(data), dtype=np.float32)
        return [dtree.train_tree_np(inp, max_depth, min_samples_per_node)]    
    return _cpu_work

In [4]:
def predict(sample, trees):
    counter = 0
    for tree in trees:
        counter += dtree.predict(sample, tree)
    
    return 1 if counter > len(trees) / 2 else 0

def evaluar(dataset, trees):
    aciertos = 0
    for sample in dataset:
        etiqueta = predict(sample, trees)
        if int(etiqueta) == int(sample[-1]):
            aciertos += 1
        
    return aciertos / dataset.shape[0]


In [5]:
# 4. Damos el número de particiones deseadas en el RDD para trabajar con el modelo
def eval_model(rdd, repeats, max_depth=4, seed=0):
    (trainingData, testData) = rdd.randomSplit([0.9, 0.1], seed=seed)
    trainingData = trainingData.cache()
    
    # Una primera vez que no medimos el tiempo
    inicio = time.time()
    arboles = trainingData.mapPartitions(gpu_work(max_depth=max_depth))
    arboles = arboles.collect()
    fin = time.time()
    print('Primera vez', fin - inicio)
    
    # Las repeat siguientes medimos el tiempo
    for i in range(repeats):
        inicio = time.time()
        arboles = trainingData.mapPartitions(gpu_work(max_depth=max_depth))
        arboles = arboles.collect()
        fin = time.time()
        print('Tiempo utilizado [GPU]:', fin-inicio)

    # Evaluamos la precision
    precision = evaluar(np.float32(testData.collect()), arboles) * 100
    print('Precisión [GPU]', precision)
    
    for i in range(repeats):
        inicio = time.time()
        arboles = trainingData.mapPartitions(cpu_work(max_depth=max_depth))
        arboles = arboles.collect()
        fin = time.time()
        print('Tiempo utilizado [CPU]:', fin-inicio)

    # Evaluamos la precision
    precision = evaluar(np.float32(testData.collect()), arboles) * 100
    print('Precisión [CPU]', precision)
    trainingData.unpersist()
    testData.unpersist()
    
    
    

In [6]:
eval_model(rdd, repeats=5, max_depth=4, seed=0)

Primera vez 102.89843511581421
Tiempo utilizado [GPU]: 25.674546480178833
Tiempo utilizado [GPU]: 25.626336097717285
Tiempo utilizado [GPU]: 25.713415384292603
Tiempo utilizado [GPU]: 25.69814705848694
Tiempo utilizado [GPU]: 25.418145656585693
Precisión [GPU] 75.91173974993197
Tiempo utilizado [CPU]: 27.806687593460083
Tiempo utilizado [CPU]: 27.729249715805054
Tiempo utilizado [CPU]: 27.07365322113037
Tiempo utilizado [CPU]: 28.169651985168457
Tiempo utilizado [CPU]: 27.57110595703125
Precisión [CPU] 75.91173974993197


In [7]:
eval_model(rdd, repeats=5, max_depth=4, seed=1)

Primera vez 32.11091208457947
Tiempo utilizado [GPU]: 25.460184812545776
Tiempo utilizado [GPU]: 25.434159994125366
Tiempo utilizado [GPU]: 25.67987322807312
Tiempo utilizado [GPU]: 25.554270029067993
Tiempo utilizado [GPU]: 25.234865427017212
Precisión [GPU] 75.90468738430629
Tiempo utilizado [CPU]: 27.786797046661377
Tiempo utilizado [CPU]: 27.26883101463318
Tiempo utilizado [CPU]: 27.78830361366272
Tiempo utilizado [CPU]: 27.268831729888916
Tiempo utilizado [CPU]: 27.535074472427368
Precisión [CPU] 75.90468738430629


In [8]:
eval_model(rdd, repeats=5, max_depth=4, seed=2)

Primera vez 32.40732669830322
Tiempo utilizado [GPU]: 25.802496194839478
Tiempo utilizado [GPU]: 26.048721075057983
Tiempo utilizado [GPU]: 26.063733100891113
Tiempo utilizado [GPU]: 25.472195386886597
Tiempo utilizado [GPU]: 25.55871844291687
Precisión [GPU] 75.7282369421653
Tiempo utilizado [CPU]: 27.69290542602539
Tiempo utilizado [CPU]: 27.534072399139404
Tiempo utilizado [CPU]: 27.702225923538208
Tiempo utilizado [CPU]: 27.733253717422485
Tiempo utilizado [CPU]: 27.627158164978027
Precisión [CPU] 75.7282369421653


In [9]:
eval_model(rdd, repeats=5, max_depth=4, seed=3)

Primera vez 32.39765024185181
Tiempo utilizado [GPU]: 25.113869667053223
Tiempo utilizado [GPU]: 25.588300228118896
Tiempo utilizado [GPU]: 26.000659227371216
Tiempo utilizado [GPU]: 25.389909267425537
Tiempo utilizado [GPU]: 25.645572662353516
Precisión [GPU] 75.82915246639463
Tiempo utilizado [CPU]: 28.03356146812439
Tiempo utilizado [CPU]: 27.78029775619507
Tiempo utilizado [CPU]: 26.913508415222168
Tiempo utilizado [CPU]: 27.56510853767395
Tiempo utilizado [CPU]: 27.772289752960205
Precisión [CPU] 75.82915246639463


In [10]:
eval_model(rdd, repeats=5, max_depth=4, seed=4)

Primera vez 32.16852259635925
Tiempo utilizado [GPU]: 25.801494598388672
Tiempo utilizado [GPU]: 25.08982563018799
Tiempo utilizado [GPU]: 25.971300840377808
Tiempo utilizado [GPU]: 25.345078706741333
Tiempo utilizado [GPU]: 25.641350507736206
Precisión [GPU] 75.7834313684341
Tiempo utilizado [CPU]: 27.608101844787598
Tiempo utilizado [CPU]: 28.95736861228943
Tiempo utilizado [CPU]: 29.043447494506836
Tiempo utilizado [CPU]: 28.024519681930542
Tiempo utilizado [CPU]: 27.6191508769989
Precisión [CPU] 75.7834313684341


In [11]:
eval_model(rdd, repeats=5, max_depth=4, seed=5)

Primera vez 32.91188859939575
Tiempo utilizado [GPU]: 25.370102167129517
Tiempo utilizado [GPU]: 25.736434936523438
Tiempo utilizado [GPU]: 25.688393354415894
Tiempo utilizado [GPU]: 25.127471446990967
Tiempo utilizado [GPU]: 25.25763702392578
Precisión [GPU] 75.67169996102612
Tiempo utilizado [CPU]: 27.684842348098755
Tiempo utilizado [CPU]: 27.843355417251587
Tiempo utilizado [CPU]: 29.135285139083862
Tiempo utilizado [CPU]: 27.60412836074829
Tiempo utilizado [CPU]: 27.355909824371338
Precisión [CPU] 75.67169996102612


In [12]:
eval_model(rdd, repeats=5, max_depth=4, seed=6)

Primera vez 32.175182580947876
Tiempo utilizado [GPU]: 25.66537117958069
Tiempo utilizado [GPU]: 25.6233332157135
Tiempo utilizado [GPU]: 25.501230239868164
Tiempo utilizado [GPU]: 25.524723768234253
Tiempo utilizado [GPU]: 25.355088233947754
Precisión [GPU] 75.7264205624288
Tiempo utilizado [CPU]: 27.0171902179718
Tiempo utilizado [CPU]: 27.689213514328003
Tiempo utilizado [CPU]: 27.64517641067505
Tiempo utilizado [CPU]: 27.629157304763794
Tiempo utilizado [CPU]: 27.32588291168213
Precisión [CPU] 75.7264205624288


In [14]:
eval_model(rdd, repeats=5, max_depth=4, seed=7)

Primera vez 32.26472735404968
Tiempo utilizado [GPU]: 25.4580557346344
Tiempo utilizado [GPU]: 25.39612650871277
Tiempo utilizado [GPU]: 25.455214977264404
Tiempo utilizado [GPU]: 25.71641778945923
Tiempo utilizado [GPU]: 26.164843797683716
Precisión [GPU] 76.33980501025314
Tiempo utilizado [CPU]: 27.611056089401245
Tiempo utilizado [CPU]: 27.84435534477234
Tiempo utilizado [CPU]: 27.852364778518677
Tiempo utilizado [CPU]: 27.529206037521362
Tiempo utilizado [CPU]: 27.072651863098145
Precisión [CPU] 76.33980501025314


In [15]:
eval_model(rdd, repeats=5, max_depth=4, seed=8)

Primera vez 32.3108913898468
Tiempo utilizado [GPU]: 25.84353280067444
Tiempo utilizado [GPU]: 25.668373823165894
Tiempo utilizado [GPU]: 25.5512752532959
Tiempo utilizado [GPU]: 25.70539927482605
Tiempo utilizado [GPU]: 25.250914812088013
Precisión [GPU] 75.65367217748141
Tiempo utilizado [CPU]: 27.68384313583374
Tiempo utilizado [CPU]: 27.68721318244934
Tiempo utilizado [CPU]: 27.55403757095337
Tiempo utilizado [CPU]: 26.7953999042511
Tiempo utilizado [CPU]: 26.562196016311646
Precisión [CPU] 75.65367217748141


In [16]:
eval_model(rdd, repeats=5, max_depth=4, seed=9)

Primera vez 32.28126072883606
Tiempo utilizado [GPU]: 25.57777214050293
Tiempo utilizado [GPU]: 25.799936294555664
Tiempo utilizado [GPU]: 25.672933101654053
Tiempo utilizado [GPU]: 25.736435890197754
Tiempo utilizado [GPU]: 25.479201793670654
Precisión [GPU] 75.69269823502368
Tiempo utilizado [CPU]: 27.51003670692444
Tiempo utilizado [CPU]: 27.230796813964844
Tiempo utilizado [CPU]: 26.872491121292114
Tiempo utilizado [CPU]: 27.164743661880493
Tiempo utilizado [CPU]: 27.515378713607788
Precisión [CPU] 75.69269823502368


In [17]:
eval_model(rdd, repeats=5, max_depth=5, seed=0)

Primera vez 33.049731731414795
Tiempo utilizado [GPU]: 25.451175689697266
Tiempo utilizado [GPU]: 25.62233281135559
Tiempo utilizado [GPU]: 25.603679895401
Tiempo utilizado [GPU]: 25.58980679512024
Tiempo utilizado [GPU]: 25.594306707382202
Precisión [GPU] 77.2677425035621
Tiempo utilizado [CPU]: 32.10027360916138
Tiempo utilizado [CPU]: 30.90013813972473
Tiempo utilizado [CPU]: 30.95919179916382
Tiempo utilizado [CPU]: 30.471742868423462
Tiempo utilizado [CPU]: 31.321521282196045
Precisión [CPU] 77.2677425035621


In [18]:
eval_model(rdd, repeats=5, max_depth=5, seed=1)

Primera vez 33.299933195114136
Tiempo utilizado [GPU]: 25.23314094543457
Tiempo utilizado [GPU]: 25.762375593185425
Tiempo utilizado [GPU]: 25.36409592628479
Tiempo utilizado [GPU]: 25.606261491775513
Tiempo utilizado [GPU]: 25.800485610961914
Precisión [GPU] 77.03936138066011
Tiempo utilizado [CPU]: 31.74946880340576
Tiempo utilizado [CPU]: 31.08530569076538
Tiempo utilizado [CPU]: 31.277482748031616
Tiempo utilizado [CPU]: 31.482667207717896
Tiempo utilizado [CPU]: 31.359558820724487
Precisión [CPU] 77.03936138066011


In [19]:
eval_model(rdd, repeats=5, max_depth=5, seed=2)

Primera vez 33.86920642852783
Tiempo utilizado [GPU]: 26.843444347381592
Tiempo utilizado [GPU]: 26.191850900650024
Tiempo utilizado [GPU]: 25.678391218185425
Tiempo utilizado [GPU]: 25.375107288360596
Tiempo utilizado [GPU]: 25.585289478302002
Precisión [GPU] 76.88493095857515
Tiempo utilizado [CPU]: 31.433504104614258
Tiempo utilizado [CPU]: 31.731894969940186
Tiempo utilizado [CPU]: 30.97420573234558
Tiempo utilizado [CPU]: 31.73489785194397
Tiempo utilizado [CPU]: 31.71586561203003
Precisión [CPU] 76.88493095857515


In [20]:
eval_model(rdd, repeats=5, max_depth=5, seed=3)

Primera vez 34.02461886405945
Tiempo utilizado [GPU]: 25.989896774291992
Tiempo utilizado [GPU]: 26.252906560897827
Tiempo utilizado [GPU]: 25.925608158111572
Tiempo utilizado [GPU]: 25.678405284881592
Tiempo utilizado [GPU]: 25.96075987815857
Precisión [GPU] 76.9684428763831
Tiempo utilizado [CPU]: 32.75079321861267
Tiempo utilizado [CPU]: 31.537718772888184
Tiempo utilizado [CPU]: 31.736900568008423
Tiempo utilizado [CPU]: 32.11724543571472
Tiempo utilizado [CPU]: 31.840006113052368
Precisión [CPU] 76.9684428763831


In [21]:
eval_model(rdd, repeats=5, max_depth=5, seed=4)

Primera vez 32.931315898895264
Tiempo utilizado [GPU]: 26.355873346328735
Tiempo utilizado [GPU]: 26.312336921691895
Tiempo utilizado [GPU]: 26.76500368118286
Tiempo utilizado [GPU]: 26.175837516784668
Tiempo utilizado [GPU]: 26.41905689239502
Precisión [GPU] 76.95065866396446
Tiempo utilizado [CPU]: 33.077974796295166
Tiempo utilizado [CPU]: 31.759921312332153
Tiempo utilizado [CPU]: 31.76392436027527
Tiempo utilizado [CPU]: 31.8580105304718
Tiempo utilizado [CPU]: 32.08921980857849
Precisión [CPU] 76.95065866396446


In [22]:
eval_model(rdd, repeats=5, max_depth=5, seed=5)

Primera vez 32.913734436035156
Tiempo utilizado [GPU]: 26.05672788619995
Tiempo utilizado [GPU]: 26.17183256149292
Tiempo utilizado [GPU]: 26.094762325286865
Tiempo utilizado [GPU]: 25.99667239189148
Tiempo utilizado [GPU]: 26.145808696746826
Precisión [GPU] 76.90807159203334
Tiempo utilizado [CPU]: 32.169777154922485
Tiempo utilizado [CPU]: 32.66274356842041
Tiempo utilizado [CPU]: 32.44754695892334
Tiempo utilizado [CPU]: 32.37348031997681
Tiempo utilizado [CPU]: 32.34545421600342
Precisión [CPU] 76.90787172594362


In [23]:
eval_model(rdd, repeats=5, max_depth=5, seed=6)

Primera vez 33.07403016090393
Tiempo utilizado [GPU]: 26.18806552886963
Tiempo utilizado [GPU]: 26.156819343566895
Tiempo utilizado [GPU]: 25.867554664611816
Tiempo utilizado [GPU]: 26.197856426239014
Tiempo utilizado [GPU]: 26.249903678894043
Precisión [GPU] 76.90544240801071
Tiempo utilizado [CPU]: 31.66985034942627
Tiempo utilizado [CPU]: 32.041178941726685
Tiempo utilizado [CPU]: 32.01613974571228
Tiempo utilizado [CPU]: 31.950101375579834
Tiempo utilizado [CPU]: 31.887028455734253
Precisión [CPU] 76.90544240801071


In [24]:
eval_model(rdd, repeats=5, max_depth=5, seed=7)

Primera vez 33.880786657333374
Tiempo utilizado [GPU]: 25.943623781204224
Tiempo utilizado [GPU]: 25.725425958633423
Tiempo utilizado [GPU]: 26.025699377059937
Tiempo utilizado [GPU]: 26.106899738311768
Tiempo utilizado [GPU]: 25.910594701766968
Precisión [GPU] 77.27977710889125
Tiempo utilizado [CPU]: 30.97204279899597
Tiempo utilizado [CPU]: 31.254461765289307
Tiempo utilizado [CPU]: 31.416606903076172
Tiempo utilizado [CPU]: 31.07029390335083
Tiempo utilizado [CPU]: 31.887036561965942
Precisión [CPU] 77.27977710889125


In [25]:
eval_model(rdd, repeats=5, max_depth=5, seed=8)

Primera vez 32.69866418838501
Tiempo utilizado [GPU]: 26.01869249343872
Tiempo utilizado [GPU]: 25.798500776290894
Tiempo utilizado [GPU]: 26.301564693450928
Tiempo utilizado [GPU]: 26.003679275512695
Tiempo utilizado [GPU]: 25.71134662628174
Precisión [GPU] 76.84458383054594
Tiempo utilizado [CPU]: 31.985620975494385
Tiempo utilizado [CPU]: 31.104322910308838
Tiempo utilizado [CPU]: 31.02625298500061
Tiempo utilizado [CPU]: 31.19840955734253
Tiempo utilizado [CPU]: 31.47265934944153
Precisión [CPU] 76.84438394607916


In [26]:
eval_model(rdd, repeats=5, max_depth=5, seed=9)

Primera vez 33.56046175956726
Tiempo utilizado [GPU]: 27.200769901275635
Tiempo utilizado [GPU]: 27.13756799697876
Tiempo utilizado [GPU]: 27.16877555847168
Tiempo utilizado [GPU]: 26.637255668640137
Tiempo utilizado [GPU]: 26.946539640426636
Precisión [GPU] 76.88060924663695
Tiempo utilizado [CPU]: 32.25084137916565
Tiempo utilizado [CPU]: 32.09429883956909
Tiempo utilizado [CPU]: 31.601776838302612
Tiempo utilizado [CPU]: 31.77832007408142
Tiempo utilizado [CPU]: 31.89404296875
Precisión [CPU] 76.88060924663695


In [27]:
eval_model(rdd, repeats=5, max_depth=6, seed=0)

Primera vez 32.89678621292114
Tiempo utilizado [GPU]: 26.379009246826172
Tiempo utilizado [GPU]: 26.498128652572632
Tiempo utilizado [GPU]: 26.97056007385254
Tiempo utilizado [GPU]: 26.709322214126587
Tiempo utilizado [GPU]: 26.435070276260376
Precisión [GPU] 77.3738052927333
Tiempo utilizado [CPU]: 35.80617070198059
Tiempo utilizado [CPU]: 35.54436755180359
Tiempo utilizado [CPU]: 35.7205274105072
Tiempo utilizado [CPU]: 36.903605461120605
Tiempo utilizado [CPU]: 36.92268753051758
Precisión [CPU] 77.37360517426316


In [28]:
eval_model(rdd, repeats=5, max_depth=6, seed=1)

Primera vez 33.65185022354126
Tiempo utilizado [GPU]: 26.488120317459106
Tiempo utilizado [GPU]: 26.67228865623474
Tiempo utilizado [GPU]: 26.58083200454712
Tiempo utilizado [GPU]: 26.203861713409424
Tiempo utilizado [GPU]: 26.590213298797607
Precisión [GPU] 77.43139424818341
Tiempo utilizado [CPU]: 35.660409688949585
Tiempo utilizado [CPU]: 35.833629846572876
Tiempo utilizado [CPU]: 35.52435040473938
Tiempo utilizado [CPU]: 35.70651364326477
Tiempo utilizado [CPU]: 36.2209837436676
Precisión [CPU] 77.43139424818341


In [29]:
eval_model(rdd, repeats=5, max_depth=6, seed=2)

Primera vez 33.674257040023804
Tiempo utilizado [GPU]: 26.66127896308899
Tiempo utilizado [GPU]: 26.09175944328308
Tiempo utilizado [GPU]: 25.912686109542847
Tiempo utilizado [GPU]: 25.90959334373474
Tiempo utilizado [GPU]: 26.27605700492859
Precisión [GPU] 77.3690214128477
Tiempo utilizado [CPU]: 34.722423791885376
Tiempo utilizado [CPU]: 34.508421659469604
Tiempo utilizado [CPU]: 34.59149980545044
Tiempo utilizado [CPU]: 35.403239011764526
Tiempo utilizado [CPU]: 34.463382720947266
Precisión [CPU] 77.3690214128477


In [30]:
eval_model(rdd, repeats=5, max_depth=6, seed=3)

Primera vez 32.92789006233215
Tiempo utilizado [GPU]: 26.365016222000122
Tiempo utilizado [GPU]: 26.18506908416748
Tiempo utilizado [GPU]: 25.885579586029053
Tiempo utilizado [GPU]: 26.853453874588013
Tiempo utilizado [GPU]: 26.46108055114746
Precisión [GPU] 77.42672059267916
Tiempo utilizado [CPU]: 35.40167856216431
Tiempo utilizado [CPU]: 35.64545822143555
Tiempo utilizado [CPU]: 35.6624755859375
Tiempo utilizado [CPU]: 35.0008704662323
Tiempo utilizado [CPU]: 35.32316708564758
Precisión [CPU] 77.42672059267916


In [31]:
eval_model(rdd, repeats=5, max_depth=6, seed=4)

Primera vez 33.18377614021301
Tiempo utilizado [GPU]: 25.59630799293518
Tiempo utilizado [GPU]: 26.11812734603882
Tiempo utilizado [GPU]: 26.08575439453125
Tiempo utilizado [GPU]: 26.016690731048584
Tiempo utilizado [GPU]: 25.77847409248352
Precisión [GPU] 77.48650393439543
Tiempo utilizado [CPU]: 35.148037910461426
Tiempo utilizado [CPU]: 35.15100908279419
Tiempo utilizado [CPU]: 35.14600396156311
Tiempo utilizado [CPU]: 35.20806074142456
Tiempo utilizado [CPU]: 34.80943417549133
Precisión [CPU] 77.48650393439543


In [32]:
eval_model(rdd, repeats=5, max_depth=6, seed=5)

Primera vez 34.51730298995972
Tiempo utilizado [GPU]: 26.065736293792725
Tiempo utilizado [GPU]: 26.00294828414917
Tiempo utilizado [GPU]: 25.905510663986206
Tiempo utilizado [GPU]: 25.871559381484985
Tiempo utilizado [GPU]: 25.84253215789795
Precisión [GPU] 77.25963604385062
Tiempo utilizado [CPU]: 34.59854602813721
Tiempo utilizado [CPU]: 35.48331141471863
Tiempo utilizado [CPU]: 35.11897945404053
Tiempo utilizado [CPU]: 35.61343050003052
Tiempo utilizado [CPU]: 35.10696816444397
Precisión [CPU] 77.25963604385062


In [33]:
eval_model(rdd, repeats=5, max_depth=6, seed=6)

Primera vez 33.650551080703735
Tiempo utilizado [GPU]: 28.470446586608887
Tiempo utilizado [GPU]: 41.4528489112854
Tiempo utilizado [GPU]: 26.129793167114258
Tiempo utilizado [GPU]: 25.885587692260742
Tiempo utilizado [GPU]: 25.932687759399414
Precisión [GPU] 77.70111726260667
Tiempo utilizado [CPU]: 35.82424855232239
Tiempo utilizado [CPU]: 35.355337381362915
Tiempo utilizado [CPU]: 35.80560612678528
Tiempo utilizado [CPU]: 35.549371004104614
Tiempo utilizado [CPU]: 34.5985050201416
Precisión [CPU] 77.70111726260667


In [34]:
eval_model(rdd, repeats=5, max_depth=6, seed=7)

Primera vez 33.33692646026611
Tiempo utilizado [GPU]: 26.12936758995056
Tiempo utilizado [GPU]: 26.227856874465942
Tiempo utilizado [GPU]: 26.174476146697998
Tiempo utilizado [GPU]: 25.874972820281982
Tiempo utilizado [GPU]: 25.90057396888733
Precisión [GPU] 77.58417377192036
Tiempo utilizado [CPU]: 35.47934699058533
Tiempo utilizado [CPU]: 34.77066230773926
Tiempo utilizado [CPU]: 35.38522148132324
Tiempo utilizado [CPU]: 35.087952852249146
Tiempo utilizado [CPU]: 34.600507974624634
Precisión [CPU] 77.58397390543117


In [35]:
eval_model(rdd, repeats=5, max_depth=6, seed=8)

Primera vez 33.11093258857727
Tiempo utilizado [GPU]: 25.980981826782227
Tiempo utilizado [GPU]: 25.731431007385254
Tiempo utilizado [GPU]: 25.907140731811523
Tiempo utilizado [GPU]: 25.892425298690796
Tiempo utilizado [GPU]: 26.029703378677368
Precisión [GPU] 77.36388367523571
Tiempo utilizado [CPU]: 35.46436357498169
Tiempo utilizado [CPU]: 35.462292194366455
Tiempo utilizado [CPU]: 34.91078996658325
Tiempo utilizado [CPU]: 35.068934202194214
Tiempo utilizado [CPU]: 34.28922367095947
Precisión [CPU] 77.36368379076893


In [36]:
eval_model(rdd, repeats=5, max_depth=6, seed=9)

Primera vez 33.223084926605225
Tiempo utilizado [GPU]: 25.89357876777649
Tiempo utilizado [GPU]: 26.362005710601807
Tiempo utilizado [GPU]: 25.998674631118774
Tiempo utilizado [GPU]: 25.861385583877563
Tiempo utilizado [GPU]: 25.883479118347168
Precisión [GPU] 77.43588718543245
Tiempo utilizado [CPU]: 35.200605154037476
Tiempo utilizado [CPU]: 35.1372275352478
Tiempo utilizado [CPU]: 34.81970715522766
Tiempo utilizado [CPU]: 34.98585867881775
Tiempo utilizado [CPU]: 35.411245584487915
Precisión [CPU] 77.4356873013652


In [37]:
eval_model(rdd, repeats=5, max_depth=7, seed=0)

Primera vez 33.77588391304016
Tiempo utilizado [GPU]: 26.481359004974365
Tiempo utilizado [GPU]: 26.51542639732361
Tiempo utilizado [GPU]: 26.127792358398438
Tiempo utilizado [GPU]: 27.045628309249878
Tiempo utilizado [GPU]: 26.597219705581665
Precisión [GPU] 78.34117797736259
Tiempo utilizado [CPU]: 39.32603406906128
Tiempo utilizado [CPU]: 38.91243386268616
Tiempo utilizado [CPU]: 38.41898703575134
Tiempo utilizado [CPU]: 38.815343379974365
Tiempo utilizado [CPU]: 38.83736610412598
Precisión [CPU] 78.34097785889247


In [38]:
eval_model(rdd, repeats=5, max_depth=7, seed=1)

Primera vez 34.78399896621704
Tiempo utilizado [GPU]: 27.087666749954224
Tiempo utilizado [GPU]: 26.65226984024048
Tiempo utilizado [GPU]: 26.5992214679718
Tiempo utilizado [GPU]: 26.168829441070557
Tiempo utilizado [GPU]: 26.688302516937256
Precisión [GPU] 78.38916316291878
Tiempo utilizado [CPU]: 38.54008412361145
Tiempo utilizado [CPU]: 38.753289461135864
Tiempo utilizado [CPU]: 38.393962383270264
Tiempo utilizado [CPU]: 38.41998624801636
Tiempo utilizado [CPU]: 38.219804525375366
Precisión [CPU] 78.38816256856573


In [39]:
eval_model(rdd, repeats=5, max_depth=7, seed=2)

Primera vez 36.547587871551514
Tiempo utilizado [GPU]: 26.280932188034058
Tiempo utilizado [GPU]: 26.506136417388916
Tiempo utilizado [GPU]: 26.797402381896973
Tiempo utilizado [GPU]: 26.30647039413452
Tiempo utilizado [GPU]: 26.350995540618896
Precisión [GPU] 78.379627776666
Tiempo utilizado [CPU]: 39.00795531272888
Tiempo utilizado [CPU]: 38.2509925365448
Tiempo utilizado [CPU]: 38.28602409362793
Tiempo utilizado [CPU]: 39.277931690216064
Tiempo utilizado [CPU]: 38.385072231292725
Precisión [CPU] 78.379627776666


In [40]:
eval_model(rdd, repeats=5, max_depth=7, seed=3)

Primera vez 33.40709352493286
Tiempo utilizado [GPU]: 26.500242710113525
Tiempo utilizado [GPU]: 26.453199863433838
Tiempo utilizado [GPU]: 26.576570987701416
Tiempo utilizado [GPU]: 26.574310541152954
Tiempo utilizado [GPU]: 26.720970630645752
Precisión [GPU] 78.33947372107497
Tiempo utilizado [CPU]: 38.32201862335205
Tiempo utilizado [CPU]: 38.0387966632843
Tiempo utilizado [CPU]: 38.34207630157471
Tiempo utilizado [CPU]: 38.615325689315796
Tiempo utilizado [CPU]: 38.56828188896179
Precisión [CPU] 78.33967384234846


In [41]:
eval_model(rdd, repeats=5, max_depth=7, seed=4)

Primera vez 33.01846647262573
Tiempo utilizado [GPU]: 26.451197385787964
Tiempo utilizado [GPU]: 26.47622060775757
Tiempo utilizado [GPU]: 26.62135148048401
Tiempo utilizado [GPU]: 25.83535885810852
Tiempo utilizado [GPU]: 26.282042503356934
Precisión [GPU] 78.15925857082611
Tiempo utilizado [CPU]: 38.33716797828674
Tiempo utilizado [CPU]: 39.00161027908325
Tiempo utilizado [CPU]: 37.92865324020386
Tiempo utilizado [CPU]: 38.267961502075195
Tiempo utilizado [CPU]: 37.927649974823
Precisión [CPU] 78.15925857082611


In [42]:
eval_model(rdd, repeats=5, max_depth=7, seed=5)

Primera vez 33.31065130233765
Tiempo utilizado [GPU]: 26.51021957397461
Tiempo utilizado [GPU]: 26.47919201850891
Tiempo utilizado [GPU]: 26.58328628540039
Tiempo utilizado [GPU]: 26.143885850906372
Tiempo utilizado [GPU]: 26.44616150856018
Precisión [GPU] 78.25676796546314
Tiempo utilizado [CPU]: 38.76975083351135
Tiempo utilizado [CPU]: 38.28998160362244
Tiempo utilizado [CPU]: 38.44712567329407
Tiempo utilizado [CPU]: 38.227925300598145
Tiempo utilizado [CPU]: 38.19889807701111
Precisión [CPU] 78.30413622872675


In [43]:
eval_model(rdd, repeats=5, max_depth=7, seed=6)

Primera vez 33.351295709609985
Tiempo utilizado [GPU]: 26.610120058059692
Tiempo utilizado [GPU]: 26.102848052978516
Tiempo utilizado [GPU]: 26.18950128555298
Tiempo utilizado [GPU]: 26.37209439277649
Tiempo utilizado [GPU]: 26.217935800552368
Precisión [GPU] 78.31391281754043
Tiempo utilizado [CPU]: 38.66625499725342
Tiempo utilizado [CPU]: 37.662360191345215
Tiempo utilizado [CPU]: 38.12079429626465
Tiempo utilizado [CPU]: 38.1718385219574
Tiempo utilizado [CPU]: 38.112786054611206
Precisión [CPU] 78.31391281754043


In [44]:
eval_model(rdd, repeats=5, max_depth=7, seed=7)

Primera vez 33.58285427093506
Tiempo utilizado [GPU]: 26.94558548927307
Tiempo utilizado [GPU]: 26.892544746398926
Tiempo utilizado [GPU]: 26.80246329307556
Tiempo utilizado [GPU]: 26.363062381744385
Tiempo utilizado [GPU]: 26.203916311264038
Precisión [GPU] 78.45938912806245
Tiempo utilizado [CPU]: 38.54613184928894
Tiempo utilizado [CPU]: 38.05373167991638
Tiempo utilizado [CPU]: 37.90460419654846
Tiempo utilizado [CPU]: 38.13079571723938
Tiempo utilizado [CPU]: 39.21379733085632
Precisión [CPU] 78.45938912806245


In [45]:
eval_model(rdd, repeats=5, max_depth=7, seed=8)

Primera vez 33.384175300598145
Tiempo utilizado [GPU]: 26.32265591621399
Tiempo utilizado [GPU]: 26.444136142730713
Tiempo utilizado [GPU]: 25.749345064163208
Tiempo utilizado [GPU]: 26.81447434425354
Tiempo utilizado [GPU]: 26.17615270614624
Precisión [GPU] 78.30713847396204
Tiempo utilizado [CPU]: 38.254995346069336
Tiempo utilizado [CPU]: 38.16483235359192
Tiempo utilizado [CPU]: 39.046637535095215
Tiempo utilizado [CPU]: 37.92361354827881
Tiempo utilizado [CPU]: 37.42594504356384
Precisión [CPU] 78.30693858949527


In [46]:
eval_model(rdd, repeats=5, max_depth=7, seed=9)

Primera vez 33.45832276344299
Tiempo utilizado [GPU]: 26.565246105194092
Tiempo utilizado [GPU]: 26.37307119369507
Tiempo utilizado [GPU]: 26.16615605354309
Tiempo utilizado [GPU]: 26.11783790588379
Tiempo utilizado [GPU]: 26.40061664581299
Precisión [GPU] 78.32377221211698
Tiempo utilizado [CPU]: 38.35611033439636
Tiempo utilizado [CPU]: 38.02971315383911
Tiempo utilizado [CPU]: 38.03671669960022
Tiempo utilizado [CPU]: 38.43908333778381
Tiempo utilizado [CPU]: 37.97866368293762
Precisión [CPU] 78.32357232804974


In [47]:
eval_model(rdd, repeats=5, max_depth=8, seed=0)

Primera vez 33.91893744468689
Tiempo utilizado [GPU]: 27.199798822402954
Tiempo utilizado [GPU]: 26.910529136657715
Tiempo utilizado [GPU]: 27.44202208518982
Tiempo utilizado [GPU]: 27.548101902008057
Tiempo utilizado [GPU]: 26.89051103591919
Precisión [GPU] 78.73280982341547
Tiempo utilizado [CPU]: 41.6099648475647
Tiempo utilizado [CPU]: 42.12639904022217
Tiempo utilizado [CPU]: 41.55787968635559
Tiempo utilizado [CPU]: 41.108481645584106
Tiempo utilizado [CPU]: 41.518842935562134
Precisión [CPU] 78.7324095864752


In [48]:
eval_model(rdd, repeats=5, max_depth=8, seed=1)

Primera vez 34.2356333732605
Tiempo utilizado [GPU]: 26.916056156158447
Tiempo utilizado [GPU]: 27.322904586791992
Tiempo utilizado [GPU]: 27.22681760787964
Tiempo utilizado [GPU]: 27.65279746055603
Tiempo utilizado [GPU]: 27.343924045562744
Precisión [GPU] 78.70615145396366
Tiempo utilizado [CPU]: 41.955493688583374
Tiempo utilizado [CPU]: 40.90728712081909
Tiempo utilizado [CPU]: 41.306660413742065
Tiempo utilizado [CPU]: 41.02939963340759
Tiempo utilizado [CPU]: 41.03741455078125
Precisión [CPU] 78.70595133509305


In [49]:
eval_model(rdd, repeats=5, max_depth=8, seed=2)

Primera vez 34.66890215873718
Tiempo utilizado [GPU]: 27.51108455657959
Tiempo utilizado [GPU]: 27.56312394142151
Tiempo utilizado [GPU]: 27.462023973464966
Tiempo utilizado [GPU]: 27.382959604263306
Tiempo utilizado [GPU]: 27.186789512634277
Precisión [GPU] 78.56233740244146
Tiempo utilizado [CPU]: 41.57373023033142
Tiempo utilizado [CPU]: 41.47080087661743
Tiempo utilizado [CPU]: 41.09946322441101
Tiempo utilizado [CPU]: 41.62093687057495
Tiempo utilizado [CPU]: 40.99236512184143
Precisión [CPU] 78.56193716229738


In [50]:
eval_model(rdd, repeats=5, max_depth=8, seed=3)

Primera vez 34.051952838897705
Tiempo utilizado [GPU]: 27.24983859062195
Tiempo utilizado [GPU]: 27.536098957061768
Tiempo utilizado [GPU]: 27.428001165390015
Tiempo utilizado [GPU]: 27.101703643798828
Tiempo utilizado [GPU]: 27.48405170440674
Precisión [GPU] 78.5882244640252
Tiempo utilizado [CPU]: 42.31508708000183
Tiempo utilizado [CPU]: 42.182732820510864
Tiempo utilizado [CPU]: 41.324647426605225
Tiempo utilizado [CPU]: 41.17751336097717
Tiempo utilizado [CPU]: 41.04539227485657
Precisión [CPU] 78.5882244640252


In [51]:
eval_model(rdd, repeats=5, max_depth=8, seed=4)

Primera vez 37.99484205245972
Tiempo utilizado [GPU]: 26.941535711288452
Tiempo utilizado [GPU]: 27.10570216178894
Tiempo utilizado [GPU]: 27.077667713165283
Tiempo utilizado [GPU]: 27.213783740997314
Tiempo utilizado [GPU]: 27.086675882339478
Precisión [GPU] 78.67371799868486
Tiempo utilizado [CPU]: 41.219236612319946
Tiempo utilizado [CPU]: 42.35458540916443
Tiempo utilizado [CPU]: 42.58779811859131
Tiempo utilizado [CPU]: 40.982343435287476
Tiempo utilizado [CPU]: 40.74111723899841
Precisión [CPU] 78.67391786637246


In [52]:
eval_model(rdd, repeats=5, max_depth=8, seed=5)

Primera vez 34.634422063827515
Tiempo utilizado [GPU]: 28.6666259765625
Tiempo utilizado [GPU]: 27.62296152114868
Tiempo utilizado [GPU]: 27.722503185272217
Tiempo utilizado [GPU]: 27.629169940948486
Tiempo utilizado [GPU]: 27.61415672302246
Precisión [GPU] 78.60133710414023
Tiempo utilizado [CPU]: 41.71893310546875
Tiempo utilizado [CPU]: 41.803083181381226
Tiempo utilizado [CPU]: 40.79016089439392
Tiempo utilizado [CPU]: 41.58087968826294
Tiempo utilizado [CPU]: 41.35067129135132
Precisión [CPU] 78.60693335465238


In [53]:
eval_model(rdd, repeats=5, max_depth=8, seed=6)

Primera vez 34.611958265304565
Tiempo utilizado [GPU]: 27.2572979927063
Tiempo utilizado [GPU]: 27.0556480884552
Tiempo utilizado [GPU]: 27.02862310409546
Tiempo utilizado [GPU]: 27.09084987640381
Tiempo utilizado [GPU]: 27.16473889350891
Precisión [GPU] 78.69626046809105
Tiempo utilizado [CPU]: 42.643991470336914
Tiempo utilizado [CPU]: 41.316640853881836
Tiempo utilizado [CPU]: 41.20645499229431
Tiempo utilizado [CPU]: 42.17141890525818
Tiempo utilizado [CPU]: 42.02628207206726
Precisión [CPU] 78.69566086382987


In [54]:
eval_model(rdd, repeats=5, max_depth=8, seed=7)

Primera vez 34.05557179450989
Tiempo utilizado [GPU]: 27.340901851654053
Tiempo utilizado [GPU]: 27.266835927963257
Tiempo utilizado [GPU]: 26.90950918197632
Tiempo utilizado [GPU]: 27.256825923919678
Tiempo utilizado [GPU]: 27.630165338516235
Precisión [GPU] 78.79756322776386
Tiempo utilizado [CPU]: 41.99938082695007
Tiempo utilizado [CPU]: 42.37559676170349
Tiempo utilizado [CPU]: 41.628262758255005
Tiempo utilizado [CPU]: 41.59088182449341
Tiempo utilizado [CPU]: 41.951210498809814
Precisión [CPU] 78.79716349478548


In [55]:
eval_model(rdd, repeats=5, max_depth=8, seed=8)

Primera vez 36.03714466094971
Tiempo utilizado [GPU]: 27.505051136016846
Tiempo utilizado [GPU]: 27.16974687576294
Tiempo utilizado [GPU]: 27.364522695541382
Tiempo utilizado [GPU]: 27.45302128791809
Tiempo utilizado [GPU]: 27.483803749084473
Precisión [GPU] 78.57038631670893
Tiempo utilizado [CPU]: 42.1048698425293
Tiempo utilizado [CPU]: 41.85811686515808
Tiempo utilizado [CPU]: 41.88314747810364
Tiempo utilizado [CPU]: 42.302273988723755
Tiempo utilizado [CPU]: 40.57595682144165
Precisión [CPU] 78.56998654777539


In [56]:
eval_model(rdd, repeats=5, max_depth=8, seed=9)

Primera vez 34.3440899848938
Tiempo utilizado [GPU]: 27.67720913887024
Tiempo utilizado [GPU]: 27.51907253265381
Tiempo utilizado [GPU]: 27.319875240325928
Tiempo utilizado [GPU]: 27.46101999282837
Tiempo utilizado [GPU]: 27.229793548583984
Precisión [GPU] 78.73973095604549
Tiempo utilizado [CPU]: 42.652461528778076
Tiempo utilizado [CPU]: 43.047208309173584
Tiempo utilizado [CPU]: 43.344478368759155
Tiempo utilizado [CPU]: 42.38661217689514
Tiempo utilizado [CPU]: 42.41360640525818
Precisión [CPU] 78.73973095604549


In [6]:
eval_model(rdd, repeats=5, max_depth=9, seed=0)

Primera vez 111.21714162826538
Tiempo utilizado [GPU]: 29.90855622291565
Tiempo utilizado [GPU]: 29.520558834075928
Tiempo utilizado [GPU]: 29.635642290115356
Tiempo utilizado [GPU]: 29.71027159690857
Tiempo utilizado [GPU]: 29.703112363815308
Precisión [GPU] 79.02658373757264
Tiempo utilizado [CPU]: 45.80850267410278
Tiempo utilizado [CPU]: 45.27508330345154
Tiempo utilizado [CPU]: 44.61044931411743
Tiempo utilizado [CPU]: 45.67246055603027
Tiempo utilizado [CPU]: 45.66445326805115
Precisión [CPU] 79.02658373757264


In [6]:
eval_model(rdd, repeats=5, max_depth=9, seed=1)

Primera vez 111.50152087211609
Tiempo utilizado [GPU]: 30.229514598846436
Tiempo utilizado [GPU]: 29.750093936920166
Tiempo utilizado [GPU]: 30.04035973548889
Tiempo utilizado [GPU]: 30.37766718864441
Tiempo utilizado [GPU]: 30.047366619110107
Precisión [GPU] 79.09037968553321
Tiempo utilizado [CPU]: 46.757750272750854
Tiempo utilizado [CPU]: 46.870697021484375
Tiempo utilizado [CPU]: 45.72064924240112
Tiempo utilizado [CPU]: 46.36121582984924
Tiempo utilizado [CPU]: 46.9285774230957
Precisión [CPU] 79.08977932892138


In [9]:
eval_model(rdd, repeats=5, max_depth=9, seed=2)

Primera vez 60.69128918647766
Tiempo utilizado [GPU]: 46.34767770767212
Tiempo utilizado [GPU]: 30.189589738845825
Tiempo utilizado [GPU]: 29.14372444152832
Tiempo utilizado [GPU]: 28.884397506713867
Tiempo utilizado [GPU]: 29.564018726348877
Precisión [GPU] 78.98899339603761
Tiempo utilizado [CPU]: 46.035539627075195
Tiempo utilizado [CPU]: 44.78847932815552
Tiempo utilizado [CPU]: 44.03122901916504
Tiempo utilizado [CPU]: 44.32142186164856
Tiempo utilizado [CPU]: 44.72077751159668
Precisión [CPU] 78.98959375625375


In [10]:
eval_model(rdd, repeats=5, max_depth=9, seed=3)

Primera vez 40.852370500564575
Tiempo utilizado [GPU]: 28.953405380249023
Tiempo utilizado [GPU]: 29.189621210098267
Tiempo utilizado [GPU]: 28.87287974357605
Tiempo utilizado [GPU]: 28.99344229698181
Tiempo utilizado [GPU]: 29.04348850250244
Precisión [GPU] 78.98146276643647
Tiempo utilizado [CPU]: 45.43410921096802
Tiempo utilizado [CPU]: 44.56764268875122
Tiempo utilizado [CPU]: 43.870009660720825
Tiempo utilizado [CPU]: 44.04117465019226
Tiempo utilizado [CPU]: 44.30139398574829
Precisión [CPU] 78.98306373662439


In [7]:
eval_model(rdd, repeats=5, max_depth=9, seed=4)

Primera vez 40.21311593055725
Tiempo utilizado [GPU]: 30.089404582977295
Tiempo utilizado [GPU]: 30.49177074432373
Tiempo utilizado [GPU]: 30.10441827774048
Tiempo utilizado [GPU]: 30.337631225585938
Tiempo utilizado [GPU]: 30.14945960044861
Precisión [GPU] 79.07505231536722
Tiempo utilizado [CPU]: 46.03891897201538
Tiempo utilizado [CPU]: 45.95185351371765
Tiempo utilizado [CPU]: 45.76467943191528
Tiempo utilizado [CPU]: 45.440385818481445
Tiempo utilizado [CPU]: 45.93783903121948
Precisión [CPU] 79.07445271230445


In [8]:
eval_model(rdd, repeats=5, max_depth=9, seed=5)

Primera vez 37.24574565887451
Tiempo utilizado [GPU]: 29.758102416992188
Tiempo utilizado [GPU]: 29.91024136543274
Tiempo utilizado [GPU]: 30.367730855941772
Tiempo utilizado [GPU]: 30.017338752746582
Tiempo utilizado [GPU]: 30.20851182937622
Precisión [GPU] 78.9598968690977
Tiempo utilizado [CPU]: 45.8075385093689
Tiempo utilizado [CPU]: 45.47239065170288
Tiempo utilizado [CPU]: 45.984872579574585
Tiempo utilizado [CPU]: 45.97987675666809
Tiempo utilizado [CPU]: 46.27014231681824
Precisión [CPU] 78.96129593172574


In [9]:
eval_model(rdd, repeats=5, max_depth=9, seed=6)

Primera vez 39.85251832008362
Tiempo utilizado [GPU]: 30.235538959503174
Tiempo utilizado [GPU]: 30.156464338302612
Tiempo utilizado [GPU]: 30.01733899116516
Tiempo utilizado [GPU]: 30.632899045944214
Tiempo utilizado [GPU]: 30.30860424041748
Precisión [GPU] 78.99126576459537
Tiempo utilizado [CPU]: 45.73592257499695
Tiempo utilizado [CPU]: 45.75860571861267
Tiempo utilizado [CPU]: 45.69753432273865
Tiempo utilizado [CPU]: 45.847678661346436
Tiempo utilizado [CPU]: 45.862693071365356
Precisión [CPU] 78.99086602842125


In [10]:
eval_model(rdd, repeats=5, max_depth=9, seed=7)

Primera vez 37.39288401603699
Tiempo utilizado [GPU]: 30.31155514717102
Tiempo utilizado [GPU]: 29.98525881767273
Tiempo utilizado [GPU]: 30.21346640586853
Tiempo utilizado [GPU]: 30.468698501586914
Tiempo utilizado [GPU]: 30.267515897750854
Precisión [GPU] 79.08357217378791
Tiempo utilizado [CPU]: 45.62712526321411
Tiempo utilizado [CPU]: 45.7615921497345
Tiempo utilizado [CPU]: 45.59444880485535
Tiempo utilizado [CPU]: 45.64549493789673
Tiempo utilizado [CPU]: 45.77060890197754
Precisión [CPU] 79.08217310836362


In [11]:
eval_model(rdd, repeats=5, max_depth=9, seed=8)

Primera vez 42.74025201797485
Tiempo utilizado [GPU]: 30.2435085773468
Tiempo utilizado [GPU]: 30.480724811553955
Tiempo utilizado [GPU]: 30.14742088317871
Tiempo utilizado [GPU]: 30.097376346588135
Tiempo utilizado [GPU]: 30.550788402557373
Precisión [GPU] 78.95316507058921
Tiempo utilizado [CPU]: 45.908539056777954
Tiempo utilizado [CPU]: 46.25706648826599
Tiempo utilizado [CPU]: 45.94479036331177
Tiempo utilizado [CPU]: 45.478365659713745
Tiempo utilizado [CPU]: 45.46935796737671
Precisión [CPU] 78.95296518612243


In [23]:
eval_model(rdd, repeats=5, max_depth=9, seed=9)

Primera vez 37.80021929740906
Tiempo utilizado [GPU]: 30.01133394241333
Tiempo utilizado [GPU]: 30.46737027168274
Tiempo utilizado [GPU]: 30.390679121017456
Tiempo utilizado [GPU]: 30.094409227371216
Tiempo utilizado [GPU]: 29.813154458999634
Precisión [GPU] 79.05594755042075
Tiempo utilizado [CPU]: 49.75508713722229
Tiempo utilizado [CPU]: 48.61627697944641
Tiempo utilizado [CPU]: 48.26996397972107
Tiempo utilizado [CPU]: 48.368051528930664
Tiempo utilizado [CPU]: 46.86759638786316
Precisión [CPU] 79.05494813008454


In [13]:
eval_model(rdd, repeats=5, max_depth=10, seed=0)

Primera vez 42.534680128097534
Tiempo utilizado [GPU]: 34.62249970436096
Tiempo utilizado [GPU]: 34.20612359046936
Tiempo utilizado [GPU]: 33.66663408279419
Tiempo utilizado [GPU]: 34.244195222854614
Tiempo utilizado [GPU]: 33.741700649261475
Precisión [GPU] 79.2525174903543
Tiempo utilizado [CPU]: 48.79428458213806
Tiempo utilizado [CPU]: 48.861457109451294
Tiempo utilizado [CPU]: 48.556188344955444
Tiempo utilizado [CPU]: 48.619229316711426
Tiempo utilizado [CPU]: 48.713322162628174
Precisión [CPU] 79.25391831964524


In [14]:
eval_model(rdd, repeats=5, max_depth=10, seed=1)

Primera vez 51.77150845527649
Tiempo utilizado [GPU]: 33.927900314331055
Tiempo utilizado [GPU]: 33.68167686462402
Tiempo utilizado [GPU]: 33.312339544296265
Tiempo utilizado [GPU]: 33.66766309738159
Tiempo utilizado [GPU]: 33.54855537414551
Precisión [GPU] 79.27869154277641
Tiempo utilizado [CPU]: 48.1668016910553
Tiempo utilizado [CPU]: 48.668317794799805
Tiempo utilizado [CPU]: 48.6312940120697
Tiempo utilizado [CPU]: 48.94457507133484
Tiempo utilizado [CPU]: 49.31204175949097
Precisión [CPU] 79.27969213712946


In [15]:
eval_model(rdd, repeats=5, max_depth=10, seed=2)

Primera vez 40.4588942527771
Tiempo utilizado [GPU]: 33.255287885665894
Tiempo utilizado [GPU]: 33.676671743392944
Tiempo utilizado [GPU]: 33.2002375125885
Tiempo utilizado [GPU]: 33.362385511398315
Tiempo utilizado [GPU]: 33.58358860015869
Precisión [GPU] 79.21833099859916
Tiempo utilizado [CPU]: 48.54981708526611
Tiempo utilizado [CPU]: 48.46013402938843
Tiempo utilizado [CPU]: 48.54120945930481
Tiempo utilizado [CPU]: 48.36304783821106
Tiempo utilizado [CPU]: 48.20790648460388
Precisión [CPU] 79.21392835701421


In [16]:
eval_model(rdd, repeats=5, max_depth=10, seed=3)

Primera vez 41.91169762611389
Tiempo utilizado [GPU]: 33.472485303878784
Tiempo utilizado [GPU]: 33.600602865219116
Tiempo utilizado [GPU]: 33.93690848350525
Tiempo utilizado [GPU]: 33.63263177871704
Tiempo utilizado [GPU]: 33.69168519973755
Precisión [GPU] 79.25122624310332
Tiempo utilizado [CPU]: 48.101916790008545
Tiempo utilizado [CPU]: 48.7854323387146
Tiempo utilizado [CPU]: 48.60226798057556
Tiempo utilizado [CPU]: 48.25194430351257
Tiempo utilizado [CPU]: 48.7734215259552
Precisión [CPU] 79.25262709201776


In [17]:
eval_model(rdd, repeats=5, max_depth=10, seed=4)

Primera vez 40.91821360588074
Tiempo utilizado [GPU]: 33.683678150177
Tiempo utilizado [GPU]: 33.3874077796936
Tiempo utilizado [GPU]: 33.38640737533569
Tiempo utilizado [GPU]: 33.64164090156555
Tiempo utilizado [GPU]: 33.5745792388916
Precisión [GPU] 79.25913045563837
Tiempo utilizado [CPU]: 48.60301184654236
Tiempo utilizado [CPU]: 48.523191690444946
Tiempo utilizado [CPU]: 48.384066581726074
Tiempo utilizado [CPU]: 48.389071226119995
Tiempo utilizado [CPU]: 48.74639821052551
Precisión [CPU] 79.25973005870114


In [18]:
eval_model(rdd, repeats=5, max_depth=10, seed=5)

Primera vez 40.288241386413574
Tiempo utilizado [GPU]: 33.214242458343506
Tiempo utilizado [GPU]: 33.1391818523407
Tiempo utilizado [GPU]: 33.4624764919281
Tiempo utilizado [GPU]: 33.67767286300659
Tiempo utilizado [GPU]: 33.80478835105896
Precisión [GPU] 79.19793738195409
Tiempo utilizado [CPU]: 49.06868815422058
Tiempo utilizado [CPU]: 48.95858931541443
Tiempo utilizado [CPU]: 48.57524371147156
Tiempo utilizado [CPU]: 48.53019738197327
Tiempo utilizado [CPU]: 48.831475019454956
Precisión [CPU] 79.1981372480438


In [19]:
eval_model(rdd, repeats=5, max_depth=10, seed=6)

Primera vez 40.94629240036011
Tiempo utilizado [GPU]: 33.46848177909851
Tiempo utilizado [GPU]: 33.12516975402832
Tiempo utilizado [GPU]: 33.440457820892334
Tiempo utilizado [GPU]: 33.28731608390808
Tiempo utilizado [GPU]: 33.74773669242859
Precisión [GPU] 79.20272620070753
Tiempo utilizado [CPU]: 48.65431618690491
Tiempo utilizado [CPU]: 48.78943753242493
Tiempo utilizado [CPU]: 48.619279623031616
Tiempo utilizado [CPU]: 48.52719736099243
Tiempo utilizado [CPU]: 48.833476066589355
Precisión [CPU] 79.20352567305578


In [20]:
eval_model(rdd, repeats=5, max_depth=10, seed=7)

Primera vez 41.12954592704773
Tiempo utilizado [GPU]: 33.569573640823364
Tiempo utilizado [GPU]: 33.8348183631897
Tiempo utilizado [GPU]: 33.66165518760681
Tiempo utilizado [GPU]: 33.84526062011719
Tiempo utilizado [GPU]: 34.01898264884949
Precisión [GPU] 79.29822878317285
Tiempo utilizado [CPU]: 48.786648750305176
Tiempo utilizado [CPU]: 48.58725094795227
Tiempo utilizado [CPU]: 49.30290389060974
Tiempo utilizado [CPU]: 48.96859860420227
Tiempo utilizado [CPU]: 48.61928129196167
Precisión [CPU] 79.29942798210796


In [21]:
eval_model(rdd, repeats=5, max_depth=10, seed=8)

Primera vez 41.15239143371582
Tiempo utilizado [GPU]: 33.60360550880432
Tiempo utilizado [GPU]: 33.48549795150757
Tiempo utilizado [GPU]: 33.545552253723145
Tiempo utilizado [GPU]: 33.453468322753906
Tiempo utilizado [GPU]: 33.32535147666931
Precisión [GPU] 79.20801776573141
Tiempo utilizado [CPU]: 48.39300584793091
Tiempo utilizado [CPU]: 48.6423020362854
Tiempo utilizado [CPU]: 48.458133935928345
Tiempo utilizado [CPU]: 48.75340533256531
Tiempo utilizado [CPU]: 48.67733192443848
Precisión [CPU] 79.20661857446396


In [22]:
eval_model(rdd, repeats=5, max_depth=10, seed=9)

Primera vez 40.50474429130554
Tiempo utilizado [GPU]: 33.562567710876465
Tiempo utilizado [GPU]: 33.83281350135803
Tiempo utilizado [GPU]: 33.55856442451477
Tiempo utilizado [GPU]: 33.68968391418457
Tiempo utilizado [GPU]: 33.43845319747925
Precisión [GPU] 79.3691658837874
Tiempo utilizado [CPU]: 48.70733022689819
Tiempo utilizado [CPU]: 48.620280265808105
Tiempo utilizado [CPU]: 48.2038893699646
Tiempo utilizado [CPU]: 49.148762941360474
Tiempo utilizado [CPU]: 49.35395002365112
Precisión [CPU] 79.37376321733394
